# Keras Model pruning

Le pruning, ou élagage en français, c’est l’idée de réduire la taille d’un réseau de neurones, tout en minimisant la perte de performance.
La performance étant définie par :
    
    – Les métriques classiques en Machine Learning
    – Le temps d’inférence
    – Le nombre de paramètres du réseau
    – Etc..

Méthodes pour y parvenir:

    (M1) Mettre à 0 certains paramètres: "Weight Pruning"
    (M2) Mettre à 0 des neurones entiers: "Unit/Neuron pruning"

<img src="photo.png" alt="Pruning example" width="500">




# Initialisation

In [20]:
import math
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.axes as axes
import numpy as np
import os
import pandas as pd
import seaborn as sns
import tempfile
import tensorboard
import tensorflow as tf
import timeit
import zipfile

from IPython.core.pylabtools import figsize
from numpy import linalg as LA
from tensorflow.keras.models import load_model
from tensorflow_model_optimization.sparsity import keras as sparsity

# Base de données

Nous allons utilisé la base de données MNIST

In [21]:
def load_dataset(dataset='mnist'):

    # Préciser les dimentions des images
    img_rows, img_cols = 28, 28
    
    if dataset=='mnist':
        # Nombre de classe
        num_classes = 10
        # Découper les données en données d'apprentissage/testes
        (x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

    else:
        print('dataset name does not match available options \n( mnist | keras )')

    x_train = x_train.reshape(x_train.shape[0], img_rows*img_cols)
    x_test = x_test.reshape(x_test.shape[0], img_rows*img_cols)
    input_shape = (img_rows*img_cols*1,)

    x_train = x_train.astype('float32')
    x_test = x_test.astype('float32')
    x_train /= 255
    x_test /= 255
    
    print('x_train shape:', x_train.shape)
    print(x_train.shape[0], 'train samples')
    print(x_test.shape[0], 'test samples')

    # Convertir les vecteurs en matrices binaires
    y_train = tf.keras.utils.to_categorical(y_train, num_classes)
    y_test = tf.keras.utils.to_categorical(y_test, num_classes)

    return x_train, x_test, y_train, y_test, num_classes, input_shape

# On charge les données de la base MNIST. 

mnist_x_train, mnist_x_test, mnist_y_train, mnist_y_test, num_classes, input_shape = load_dataset(dataset='mnist')



x_train shape: (60000, 784)
60000 train samples
10000 test samples


# Entrainer un modèle de Keras sans élagage

Dans cette partie, nous alons construire un réseau de neurone à 4 couches denses et complétement connectées avec les tailles suivantes: 1000, 1000, 500 et 200. Une cinquième couche pour les logs de résultants (de taille 10).


In [22]:
l = tf.keras.layers


"""  Construire le modèle
    
    @args: input_shape: Décrire le format des données saisies
           num_classes: Le nombre de classe des labels
           sparsity: Le facteur de parcimonie
           
    @return: TF.Keras modèle avec 4 couches denses.



"""

def build_model_arch(input_shape, num_classes, sparsity=0.0):

    model = tf.keras.Sequential()

    model.add(l.Dense(int(1000-(1000*sparsity)), activation='relu',
                      input_shape=input_shape)),
    model.add(l.Dense(int(1000-(1000*sparsity)), activation='relu'))
    model.add(l.Dense(int(500-(500*sparsity)), activation='relu'))
    model.add(l.Dense(int(200-(200*sparsity)), activation='relu'))
    model.add(l.Dense(num_classes, activation='softmax'))

    return model


mnist_model_base = build_model_arch(input_shape, num_classes)

In [25]:
logdir = tempfile.mkdtemp()
print('Writing training logs to ' + logdir)

Writing training logs to /tmp/tmplw380ll6


# Entrainer le modèle

In [32]:
"""
Entrainer le modèle créé

@args: model: Keras modèle
      x_train: données d'entrainement
      y_train: Labels des données d'entrainement
      batch_size: la taille du batch
      epochs: Le nombre d'époque d'entrainement
      x_test: données de tests
      y_test: Labels des données de tests
      
@return: modèle entrainé + statistique (perte + précision)
"""

def make_nosparse_model(model, x_train, y_train, batch_size, 
                         epochs, x_test, y_test):
    
    callbacks = [tf.keras.callbacks.TensorBoard(log_dir=logdir, profile_batch=0)]

    model.compile(
        loss=tf.keras.losses.categorical_crossentropy,
        optimizer='adam',
        metrics=['accuracy'])

    model.fit(x_train, y_train,
              batch_size=batch_size,
              epochs=epochs,
              verbose=1,
              callbacks=callbacks,
              validation_data=(x_test, y_test))
    score = model.evaluate(x_test, y_test, verbose=0)
    print('Test loss:', score[0])
    print('Test accuracy:', score[1])
    
    return model, score

batch_size = 128
epochs = 10

mnist_model, mnist_score = make_nosparse_model(mnist_model_base,
                                               mnist_x_train,
                                               mnist_y_train,
                                               batch_size,
                                               epochs,
                                               mnist_x_test,
                                               mnist_y_test)
print(mnist_model.summary())


Epoch 1/10


2022-02-03 06:49:25.570639: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 188160000 exceeds 10% of free system memory.


469/469 [==============================] - 7s 14ms/step - loss: 0.0136 - accuracy: 0.9968 - val_loss: 0.1242 - val_accuracy: 0.9830
Epoch 2/10
469/469 [==============================] - 7s 14ms/step - loss: 0.0148 - accuracy: 0.9964 - val_loss: 0.0979 - val_accuracy: 0.9837
Epoch 3/10
469/469 [==============================] - 7s 14ms/step - loss: 0.0135 - accuracy: 0.9968 - val_loss: 0.1287 - val_accuracy: 0.9820
Epoch 4/10
469/469 [==============================] - 7s 14ms/step - loss: 0.0088 - accuracy: 0.9977 - val_loss: 0.1102 - val_accuracy: 0.9780
Epoch 5/10
469/469 [==============================] - 6s 14ms/step - loss: 0.0100 - accuracy: 0.9978 - val_loss: 0.1309 - val_accuracy: 0.9843
Epoch 6/10
469/469 [==============================] - 8s 16ms/step - loss: 0.0090 - accuracy: 0.9979 - val_loss: 0.0876 - val_accuracy: 0.9823
Epoch 7/10
469/469 [==============================] - 9s 19ms/step - loss: 0.0088 - accuracy: 0.9976 - val_loss: 0.0941 - val_accuracy: 0.9828
Epoch 8/10

In [28]:
%tensorboard --logdir={logdir}

UsageError: Line magic function `%tensorboard` not found.


# Pruning

## Pruning des couches



In [ ]:
""" Retourner les matrices creuses avec une sparsity = K """

def weight_prune_dense_layer(k_weights, b_weights, k_sparsity):

    # Copy the kernel weights and get ranked indeces of the abs
    kernel_weights = np.copy(k_weights)
    ind = np.unravel_index(
        np.argsort(
            np.abs(kernel_weights),
            axis=None),
        kernel_weights.shape)
        
    # Number of indexes to set to 0
    cutoff = int(len(ind[0])*k_sparsity)
    # The indexes in the 2D kernel weight matrix to set to 0
    sparse_cutoff_inds = (ind[0][0:cutoff], ind[1][0:cutoff])
    kernel_weights[sparse_cutoff_inds] = 0.
        
    # Copy the bias weights and get ranked indeces of the abs
    bias_weights = np.copy(b_weights)
    ind = np.unravel_index(
        np.argsort(
            np.abs(bias_weights), 
            axis=None), 
        bias_weights.shape)
        
    # Number of indexes to set to 0
    cutoff = int(len(ind[0])*k_sparsity)
    # The indexes in the 1D bias weight matrix to set to 0
    sparse_cutoff_inds = (ind[0][0:cutoff])
    bias_weights[sparse_cutoff_inds] = 0.
    
    return kernel_weights, bias_weights

In [ ]:
    """
    Takes in matrices of kernel and bias weights (for a dense
      layer) and returns the unit-pruned versions of each
@args:
      k_weights: 2D matrix 
      b_weights: 1D matrix of the biases of a dense layer
      k_sparsity: percentage of weights to set to 0
@return:
      kernel_weights: sparse matrix with same shape as the original
        kernel weight matrix
      bias_weights: sparse array with same shape as the original
        bias array
    """

    
    def unit_prune_dense_layer(k_weights, b_weights, k_sparsity):

    # Copy the kernel weights and get ranked indeces of the
    # column-wise L2 Norms
    kernel_weights = np.copy(k_weights)
    ind = np.argsort(LA.norm(kernel_weights, axis=0))
        
    # Number of indexes to set to 0
    cutoff = int(len(ind)*k_sparsity)
    # The indexes in the 2D kernel weight matrix to set to 0
    sparse_cutoff_inds = ind[0:cutoff]
    kernel_weights[:,sparse_cutoff_inds] = 0.
        
    # Copy the bias weights and get ranked indeces of the abs
    bias_weights = np.copy(b_weights)
    # The indexes in the 1D bias weight matrix to set to 0
    # Equal to the indexes of the columns that were removed in this case
    #sparse_cutoff_inds
    bias_weights[sparse_cutoff_inds] = 0.
    
    return kernel_weights, bias_weights

## Pruning sur tout un modèle

In [29]:
def sparsify_model(model, x_test, y_test, k_sparsity, pruning='weight'):
    """
    Takes in a model made of dense layers and prunes the weights
    Args:
      model: Keras model
      k_sparsity: target sparsity of the model
    Returns:
      sparse_model: sparsified copy of the previous model
    """
    # Copying a temporary sparse model from our original
    sparse_model = tf.keras.models.clone_model(model)
    sparse_model.set_weights(model.get_weights())
    
    # Getting a list of the names of each component (w + b) of each layer
    names = [weight.name for layer in sparse_model.layers for weight in layer.weights]
    # Getting the list of the weights for each component (w + b) of each layer
    weights = sparse_model.get_weights()
    
    # Initializing list that will contain the new sparse weights
    newWeightList = []

    # Iterate over all but the final 2 layers (the softmax)
    for i in range(0, len(weights)-2, 2):
        
        if pruning=='weight':
            kernel_weights, bias_weights = weight_prune_dense_layer(weights[i],
                                                                    weights[i+1],
                                                                    k_sparsity)
        elif pruning=='unit':
            kernel_weights, bias_weights = unit_prune_dense_layer(weights[i],
                                                                  weights[i+1],
                                                                  k_sparsity)
        else:
            print('does not match available pruning methods ( weight | unit )')
        
        # Append the new weight list with our sparsified kernel weights
        newWeightList.append(kernel_weights)
        
        # Append the new weight list with our sparsified bias weights
        newWeightList.append(bias_weights)

    # Adding the unchanged weights of the final 2 layers
    for i in range(len(weights)-2, len(weights)):
        unmodified_weight = np.copy(weights[i])
        newWeightList.append(unmodified_weight)

    # Setting the weights of our model to the new ones
    sparse_model.set_weights(newWeightList)
    
    # Re-compiling the Keras model (necessary for using `evaluate()`)
    sparse_model.compile(
        loss=tf.keras.losses.categorical_crossentropy,
        optimizer='adam',
        metrics=['accuracy'])
    
    # Printing the the associated loss & Accuracy for the k% sparsity
    score = sparse_model.evaluate(x_test, y_test, verbose=0)
    print('k% weight sparsity: ', k_sparsity,
          '\tTest loss: {:07.5f}'.format(score[0]),
          '\tTest accuracy: {:05.2f} %%'.format(score[1]*100.))
    
    return sparse_model, score
 

# Tester les méthodes

In [31]:
# list of sparsities
k_sparsities = [0.0, 0.25, 0.50, 0.60, 0.70, 0.80, 0.90, 0.95, 0.97, 0.99]

# The empty lists where we will store our training results
mnist_model_loss_weight = []
mnist_model_accs_weight = []
mnist_model_loss_unit = []
mnist_model_accs_unit = []
fmnist_model_loss_weight = []
fmnist_model_accs_weight = []
fmnist_model_loss_unit = []
fmnist_model_accs_unit = []

dataset = 'mnist'
pruning = 'weight'
print('\n MNIST Weight-pruning\n')
for k_sparsity in k_sparsities:
    sparse_model, score = sparsify_model(mnist_model, x_test=mnist_x_test,
                                         y_test=mnist_y_test,
                                         k_sparsity=k_sparsity, 
                                         pruning=pruning)
    mnist_model_loss_weight.append(score[0])
    mnist_model_accs_weight.append(score[1])
    
    # Save entire model to an H5 file
    sparse_model.save('models/sparse_{}-model_k-{}_{}-pruned.h5'.format(dataset, k_sparsity, pruning))
    del sparse_model


pruning='unit'
print('\n MNIST Unit-pruning\n')
for k_sparsity in k_sparsities:
    sparse_model, score = sparsify_model(mnist_model, x_test=mnist_x_test,
                                         y_test=mnist_y_test, 
                                         k_sparsity=k_sparsity, 
                                         pruning=pruning)
    mnist_model_loss_unit.append(score[0])
    mnist_model_accs_unit.append(score[1])
    
    # Save entire model to an H5 file
    sparse_model.save('models/sparse_{}-model_k-{}_{}-pruned.h5'.format(dataset, k_sparsity, pruning))
    del sparse_model



 MNIST Weight-pruning

k% weight sparsity:  0.0 	Test loss: 0.12456 	Test accuracy: 97.83 %%
k% weight sparsity:  0.25 	Test loss: 0.12211 	Test accuracy: 97.86 %%
k% weight sparsity:  0.5 	Test loss: 0.11283 	Test accuracy: 97.78 %%
k% weight sparsity:  0.6 	Test loss: 0.10753 	Test accuracy: 97.59 %%
k% weight sparsity:  0.7 	Test loss: 0.11549 	Test accuracy: 97.02 %%
k% weight sparsity:  0.8 	Test loss: 0.21914 	Test accuracy: 95.64 %%
k% weight sparsity:  0.9 	Test loss: 1.19742 	Test accuracy: 76.03 %%
k% weight sparsity:  0.95 	Test loss: 2.04617 	Test accuracy: 19.92 %%
k% weight sparsity:  0.97 	Test loss: 2.24532 	Test accuracy: 10.80 %%
k% weight sparsity:  0.99 	Test loss: 2.31206 	Test accuracy: 09.74 %%

 MNIST Unit-pruning

k% weight sparsity:  0.0 	Test loss: 0.12456 	Test accuracy: 97.83 %%
k% weight sparsity:  0.25 	Test loss: 0.10615 	Test accuracy: 97.82 %%
k% weight sparsity:  0.5 	Test loss: 0.09645 	Test accuracy: 97.98 %%
k% weight sparsity:  0.6 	Test loss: 0.

# La source du code: 
https://colab.research.google.com/drive/102oKvefYhr-jrkJqR8d0wLGJmD9gNhpd
